In [88]:
import pandas as pd
import joblib
import numpy as np
import re

# Todo

### stats, proportions of distributions, stats per quarter or 10th, words not in original dict


Data structure

mmmm, I think the final DF needs to have artist, song title, split lyrics, and sylables. Split lyrics and syllables will probably be lists.

In [89]:
'''Read in CSVs and syllable model'''

bi = pd.read_csv("../Inputs/BillieEilish.csv")
em = pd.read_csv("../Inputs/Eminem.csv")
syl = pd.read_csv("../Inputs/phoneticDictionary.csv")

syl_model = joblib.load("../../Syllable_Model/Outputs/syllable_model.pkl")

#Configure Syllable Dictionary
syl_cols_to_keep = ["word", "syl"]
syl = syl[syl_cols_to_keep]
syl_dict = {}
for i, row in syl.iterrows():
    syl_dict[row[0]] = row[1]


/var/folders/17/1lp6v8rx7hg46v35nrbf4mk00000gn/T/ipykernel_74615/2335633514.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  syl_dict[row[0]] = row[1]


In [119]:
'''Create Song class'''

class Song():
    def __init__(self,title, artist, album, lyrics, syls=None):
        self.title = title
        self.artist = artist
        self.album = album
        self.lyrics = lyrics
        self.syls = syls
    def set_syls(self,syls):
        self.syls = syls

test = Song('one step','Billie','album',['one','step','closer'])
print(test.artist)

def to_song(row):
    return Song(*row)

Billie


In [120]:
songs = []
for index,row in bi.iterrows():
    songs.append(to_song(row[['Title','Artist','Album','Lyric']]))
for song in songs:
    print(song.title)

​when the party’s over
​everything i wanted
​bad guy
​idontwannabeyouanymore
​bury a friend
​i love you
​wish you were gay
No Time To Die
​​ocean eyes
Therefore I Am
​bellyache
​you should see me in a crown
​my future
​all the good girls go to hell
​xanny
COPYCAT
​ilomilo
​listen before i go
​my strange addiction
​watch
​hostage
​my boy
Six Feet Under
8
​bitches broken hearts
​​come out and play
​​party favor
6.18.18*
​goodbye
Bored
​ocean eyes (blackbear Remix)
WHEN I WAS OLDER
The End of the World
!!!!!!!
​ocean eyes (Astronomyy Remix)
Fingers Crossed
​sHE’s brOKen
​limbo*
​hotline bling
​see-through
​another stupid song*
NOT MY RESPONSIBILITY
​true blue*
Bad
​because i’m in love with you*
​bad company / so n so*
Eres Tú
​call me back
​daddy*
I don’t know, i just wish i wasn’t ​breathing
The Hill
Sex (Cover)
​bellyache (Marian Hill Remix)
​dancing on my own
​​my boy (TroyBoi Remix)
​you don’t get me high anymore
​use me*
​good*
Thinkin Bout You (Cover)
COPYCAT (Sofi Tukker Remix)
Unr

In [121]:
songs[0].set_syls([1,1,1,2,2,3,1])
print(songs[0].syls)

[1, 1, 1, 2, 2, 3, 1]


In [90]:
print(em.head())

   Unnamed: 0  Artist          Title                              Album  \
0           0  Eminem        Rap God  The Marshall Mathers LP2 (Deluxe)   
1           1  Eminem       Killshot                                NaN   
2           2  Eminem       Godzilla            Music to Be Murdered By   
3           3  Eminem  Lose Yourself                        The Singles   
4           4  Eminem    The Monster  The Marshall Mathers LP2 (Deluxe)   

     Year        Date                                              Lyric  
0  2013.0  2013-10-15  look i was gonna go easy on you not to hurt yo...  
1  2018.0  2018-09-14  you sound like a bitch bitch shut the fuck up ...  
2  2020.0  2020-01-17  ugh you're a monster   eminem i can swallow a ...  
3  2002.0  2002-10-28  look if you had one shot or one opportunity to...  
4  2013.0  2013-10-29  rihanna i'm friends with the monster that's un...  


In [91]:
'''Define feature functions for syllable model'''

def count_vowels(word):
    vowels = 0
    for letter in word:
        if letter in ['a','e','i','o','u']:
            vowels += 1      

    return vowels

def count_vowel_groups(word):
    word = word.lower()
    groups = len(re.findall(r'[aeiouy]+', word))
    if word.endswith('e'):
        groups -= 1
    return max(groups, 1)

def word_features(word):
    vowels = count_vowels(word)
    vowel_groups = count_vowel_groups(word)
    length = len(word)
    features = pd.DataFrame(
        np.array([[length,vowels,vowel_groups]]), columns=["length", "vowel_count", "vowel_groups"])
    return features

In [92]:
'''Define row generator'''

def df_row_generator(artist,song,lyrics,dict):
    '''Take Song and Lyric info and add syllable data from dict, predict syls if word not in dict'''
    lyrics = lyrics.split()
    words = []
    syllables = []
    
    for word in lyrics:
        syls = dict.get(word)
        if syls == None:
            guess = int(syl_model.predict(word_features(word))[0][0].round())
            if guess > 8:
                continue
            syllables.append(guess)
        else:
            syllables.append(syls)
        words.append(word)

    return artist,song,words,syllables

In [93]:
'''Make Dataframe'''

df = pd.DataFrame(columns=['Artist','Title','Lyrics','Syllables'])

#Limit to single BE album
bi_short = bi.loc[bi['Album'] == 'WHEN WE ALL FALL ASLEEP, WHERE DO WE GO? (Japanese Import)']
em_short = em.loc[em['Album'] == 'The Marshall Mathers LP2 (Deluxe)']

union_df = pd.concat([bi_short, em_short])

for i, row in union_df.iterrows():
    df.loc[len(df)] = df_row_generator(row['Artist'],row['Title'],row['Lyric'],syl_dict)


In [95]:
'''Clean unicode characters'''

df['Title'] = (
    df['Title']
    .str.replace(r'[\u200b\u200c\u200d\ufeff]', '', regex=True)
)

In [96]:
df.head()

,Artist,Title,Lyrics,Syllables
0,Billie Eilish,when the party’s over,"[don't, you, know, i'm, no, good, for, you, i'...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, ..."
1,Billie Eilish,bad guy,"[white, shirt, now, red, my, bloody, nose, sle...","[1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, ..."
2,Billie Eilish,bury a friend,"[mehki, raine, billie, billie, eilish, what, d...","[2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Billie Eilish,i love you,"[it's, not, true, tell, me, ive, been, lied, t...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, ..."
4,Billie Eilish,wish you were gay,"[baby, i, don't, feel, so, good, six, words, y...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 2, 1, 1, ..."


In [97]:
'''Export DF'''

df.to_pickle("/Users/jacksonlipfert/Desktop/Projects/Lyrics Project/Data_Processing/Outputs/Lyrics_DF")